# Dense dsm with stereo-matching methods comparison

This notebook analyses the contribution of two different stereomatching methods available in Cars 



Please, let's see the sensor_to_dense_dsm_step_by_step notebook before this one

### Imports

In [ ]:
# Notebook local imports

import os
import math
import numpy as np
###
import warnings
# Filter warnings
warnings.filterwarnings("ignore",category=UserWarning)
# import pretty print
import pprint 
pp = pprint.PrettyPrinter(indent=4)

# import external function for notebook
from notebook_helpers import get_full_data, show_data, get_dir_path, set_up_demo_inputs

In [ ]:
# CARS imports

# Applications
from cars.applications.application import Application
from cars.applications.grid_generation import grid_correction_app
from cars.applications.sparse_matching import sparse_matching_algo, sparse_matching_wrappers
from cars.applications.triangulation import triangulation_algo, triangulation_wrappers
from cars.applications.dem_generation import dem_generation_constants as dem_gen_cst
from shareloc.geofunctions.rectification_grid import RectificationGrid
from cars.applications.grid_generation.transform_grid import transform_grid_func
from cars.pipelines.default.unit_pipeline import UnitPipeline


# Pipelines
import cars.pipelines.parameters.sensor_inputs_constants as sens_cst

from cars.pipelines.parameters import sensor_inputs, output_parameters
from cars.pipelines.parameters import advanced_parameters_constants as adv_cst
from cars.pipelines.pipeline_constants import ADVANCED

# Conf, core, orchestrator
from cars.core import cars_logging
from cars.core import inputs, preprocessing
from cars.core.utils import safe_makedirs
from cars.orchestrator import orchestrator
from cars.core.utils import make_relative_path_absolute
from cars import import_plugins

In [ ]:
# Show CARS version
from cars import __version__
print("CARS version used : {}".format(__version__))

In [ ]:
# Import external plugins
import_plugins()

---------

## Inputs/Outputs

### Define outputs

In [ ]:
# Modify with your own output path if needed
output_dir = os.path.join(get_dir_path(), "output_notebook")
print(output_dir)

### Define inputs

In [ ]:
# By default, the notebook use data_gizeh_small.tar.bz2, data_gizeh is available also (performance dependent).
# For you own data: Modify input_dir_path and modify all images, geometric models and color file names below
input_dir_path = set_up_demo_inputs("data_gizeh_small")

conf = {"inputs": {
    "sensors": {
        "left": {
            "image": {
              "loader": "pivot",
              "bands": {
                "b0": {
                  "path": os.path.join(input_dir_path, "img1.tif"),
                  "band": 0
                },
                "b1": {
                  "path": os.path.join(input_dir_path, "color1.tif"),
                  "band": 0
                },
                "b2": {
                  "path": os.path.join(input_dir_path, "color1.tif"),
                  "band": 1
                },
                "b3": {
                  "path": os.path.join(input_dir_path, "color1.tif"),
                  "band": 2
                },
              },
            },
            "geomodel": {
              "path": os.path.join(input_dir_path, "img1.geom")
            },
        },
        "right": {
            "image": os.path.join(input_dir_path, "img2.tif"),
            "geomodel": {
              "path": os.path.join(input_dir_path, "img2.geom")
            }
        },   
    },
    "pairing": [["left", "right"]],
},
        "output":{
            "directory": os.path.join(output_dir, "output_res4")
        },
        "advanced": {"epipolar_resolutions": 4}
}

updated_inputs_conf = sensor_inputs.sensors_check_inputs(conf["inputs"])
pp.pprint(updated_inputs_conf)

In [ ]:
# Get geometry plugin
(
    _,
    geometry_plugin_name,
    geom_plugin_without_dem_and_geoid,
    geom_plugin_with_dem_and_geoid,
    _
) = sensor_inputs.check_geometry_plugin(
    updated_inputs_conf, None
)

### Create orchestrator


In [ ]:
# Use sequential mode in notebook
orchestrator_conf = {"mode": "sequential"}
cars_orchestrator = orchestrator.Orchestrator(orchestrator_conf=orchestrator_conf, out_dir=output_dir)

#### Launch unit pipeline to get the a priori

In [ ]:
UnitPipeline(conf).run(
                    cars_orchestrator,
                    generate_dems = True,
                    which_resolution="first",
                    final_out_dir=os.path.join(output_dir, "output_res4")
                )

---------

## First, compute epipolar rectified images 

In [ ]:
dem_generation_application = Application("dem_generation")

In [ ]:
epipolar_grid_generation_application = Application("grid_generation")

In [ ]:
resampling_application = Application("resampling")

In [ ]:
sparse_matching_application = Application("sparse_matching")

#### Update_conf for the full resolution

In [ ]:
path_dem = os.path.join(output_dir, "output_res4/dump_dir/dem_generation")

conf = {"inputs": {
    "sensors": {
        "left": {
            "image": {
              "loader": "pivot",
              "bands": {
                "b0": {
                  "path": os.path.join(input_dir_path, "img1.tif"),
                  "band": 0
                },
                "b1": {
                  "path": os.path.join(input_dir_path, "color1.tif"),
                  "band": 0
                },
                "b2": {
                  "path": os.path.join(input_dir_path, "color1.tif"),
                  "band": 1
                },
                "b3": {
                  "path": os.path.join(input_dir_path, "color1.tif"),
                  "band": 2
                },
              },
            },
            "geomodel": {
              "path": os.path.join(input_dir_path, "img1.geom")
            },
        },
        "right": {
            "image": os.path.join(input_dir_path, "img2.tif"),
            "geomodel": {
              "path": os.path.join(input_dir_path, "img2.geom")
            }
        },   
    },
    "pairing": [["left", "right"]],
    "initial_elevation": os.path.join(path_dem, "dem_median.tif")
},
        "advanced":{
            "terrain_a_priori" : {
                "dem_median": os.path.join(path_dem, "dem_median.tif"),
                "dem_min": os.path.join(path_dem, "dem_min.tif"),
                "dem_max": os.path.join(path_dem, "dem_max.tif")
            }
        }
}

updated_inputs_conf = sensor_inputs.sensors_check_inputs(conf["inputs"])
pp.pprint(updated_inputs_conf)



In [ ]:
# Get geometry plugin
(
    _,
    geometry_plugin_name,
    geom_plugin_without_dem_and_geoid,
    geom_plugin_with_dem_and_geoid,
    _
) = sensor_inputs.check_geometry_plugin(
    updated_inputs_conf, None
)

### Update dem min max median

In [ ]:
dem_median = conf["advanced"]["terrain_a_priori"]["dem_median"]
dem_min = conf["advanced"]["terrain_a_priori"]["dem_min"]
dem_max = conf["advanced"]["terrain_a_priori"]["dem_max"]


### Sensors images generation

From input configuration "inputs" seen before

In [ ]:
_, sensor_image_left, sensor_image_right = sensor_inputs.generate_inputs(updated_inputs_conf, geom_plugin_without_dem_and_geoid)[0]

### Grid Generation : epipolar grid generation

In [ ]:
geom_plugin = geom_plugin_with_dem_and_geoid
if updated_inputs_conf["initial_elevation"]["dem"] is None:
    geom_plugin = geom_plugin_without_dem_and_geoid
    
grid_left, grid_right = epipolar_grid_generation_application.run(
    sensor_image_left,
    sensor_image_right,
    geom_plugin,
    orchestrator=cars_orchestrator
)

### Resampling : epipolar images generation


In [ ]:
required_bands = sparse_matching_application.get_required_bands() 

In [ ]:
epipolar_image_left, epipolar_image_right = resampling_application.run(
    sensor_image_left,
    sensor_image_right,
    grid_left,
    grid_right,
    geom_plugin,
    orchestrator=cars_orchestrator,
    margins_fun=sparse_matching_application.get_margins_fun(),
    required_bands=required_bands,
)

### Get the sift a priori 

In [ ]:

out_dir_sensor_matches = os.path.join(output_dir, "output_res4/dsm/sensor_matches/left_right")

sensor_matches_left_path = os.path.join(
    out_dir_sensor_matches,
    "sensor_matches_left.npy",
)
sensor_matches_right_path = os.path.join(
    out_dir_sensor_matches,
    "sensor_matches_right.npy",
)

sensor_matches_left = np.load(
    sensor_matches_left_path
)
sensor_matches_right = np.load(
    sensor_matches_right_path
)

new_grid_matches_array = (
    geom_plugin.transform_matches_from_grids(
        sensor_matches_left,
        sensor_matches_right,
        grid_left,
        grid_right,
    )
)

# Estimate grid_correction
(
    grid_correction_coeff,
    _,
    _,
    _,
    _,
) = grid_correction_app.estimate_right_grid_correction(
    new_grid_matches_array,
    grid_right,
    save_matches=False,
    initial_cars_ds=None,
    pair_folder=None,
    pair_key=None,
    orchestrator=cars_orchestrator,
)

# Correct grid right

corrected_grid_right = (
    grid_correction_app.correct_grid(
        grid_right,
        grid_correction_coeff,
        output_dir,
        False,
    )
)

# Use the new grid as uncorrected grid
corrected_grid_left = grid_left


#### Compute margins used in dense matching, with corresponding disparity min and max

In [ ]:
triangulation_application = Application("triangulation")

In [ ]:
dense_matching_application = Application("dense_matching")

In [ ]:
interpolated_grid_left = RectificationGrid(
    grid_left["path"],
    interpolator=geom_plugin.interpolator,
)
    
interpolated_grid_right = RectificationGrid(
    corrected_grid_right["path"],
    interpolator=geom_plugin.interpolator,
)

In [ ]:
epsg = preprocessing.compute_epsg(
    sensor_image_left, 
    sensor_image_right,
    grid_left,
    corrected_grid_right,
    geom_plugin_with_dem_and_geoid,
    disp_min=0,
    disp_max=0,
)

Compute disparity grids range (min and max) 

In [ ]:


# Generate grids
disp_range_grid = dense_matching_application.generate_disparity_grids(
    sensor_image_right,
    corrected_grid_right,
    geom_plugin_with_dem_and_geoid,
    dem_min=dem_min,
    dem_max=dem_max,
    dem_median=dem_median,
)


In [ ]:
dense_matching_margins_fun = dense_matching_application.get_margins_fun(
    corrected_grid_left, disp_range_grid)

###  Resampling: generate epipolar images with corrected grids and new margins

In [ ]:
(
    optimum_tile_size,
    local_tile_optimal_size_fun,
) = dense_matching_application.get_optimal_tile_size(
    disp_range_grid,
    cars_orchestrator.cluster.checked_conf_cluster[
        "max_ram_per_worker"
    ],
)

In [ ]:
 required_bands = dense_matching_application.get_required_bands()
# Add left required bands for texture
texture_bands = ["b1", "b2", "b3"]
required_bands["left"] = sorted(
    set(required_bands["left"]).union(set(texture_bands))
)
# Find index of texture band in left_dataset
texture_bands_indices = [
    required_bands["left"].index(band)
    for band in texture_bands
]

In [ ]:
new_epipolar_image_left, new_epipolar_image_right = resampling_application.run(
    sensor_image_left,
    sensor_image_right,
    corrected_grid_left,
    corrected_grid_right,
    geom_plugin_with_dem_and_geoid,
    orchestrator=cars_orchestrator,
    margins_fun=dense_matching_margins_fun,
    tile_width=optimum_tile_size,
    tile_height=optimum_tile_size,
    required_bands=required_bands,
)

## Dense Matching: compute disparities with pandora by using two differents methods

### Census similarity measure with semi-global matching (default method)


In [ ]:
dense_matching_census_application = Application("dense_matching")

In [ ]:
 corr_cfg = dense_matching_census_application.loader.get_conf()
left_image = conf["inputs"]["sensors"]["left"]["image"]["main_file"]
right_image = conf["inputs"]["sensors"]["right"]["image"]["main_file"]
bands_left = list(conf["inputs"]["sensors"]["left"]["image"]["bands"].keys())
bands_right = list(conf["inputs"]["sensors"]["right"]["image"]["bands"].keys())
dense_matching_census_application.corr_config = (
    dense_matching_census_application.loader.check_conf(
        corr_cfg, left_image, right_image, bands_left, bands_right
    )
)

In [ ]:
epipolar_disparity_map_census = dense_matching_census_application.run(
    new_epipolar_image_left,
    new_epipolar_image_right,
    local_tile_optimal_size_fun,
    orchestrator=cars_orchestrator,
    disp_range_grid=disp_range_grid,
    texture_bands=texture_bands_indices,
)

#### Show full disparity map

In [ ]:
data_disparity_census = get_full_data(epipolar_disparity_map_census, "disp")
show_data(data_disparity_census)

### MC-CNN, the similarity measure produced by mc-cnn neural network

<a href="https://github.com/CNES/Pandora_MCCNN">MC-CNN algorithm</a> used by <a  href="https://github.com/CNES/Pandora"> Pandora</a> as <a href="https://github.com/CNES/Pandora_plugin_mccnn">plugin</a>

In [ ]:
dense_matching_mccnn_application = Application("dense_matching", cfg={"method": "mccnn_sgm"})

In [ ]:
 corr_cfg = dense_matching_mccnn_application.loader.get_conf()
left_image = conf["inputs"]["sensors"]["left"]["image"]["main_file"]
right_image = conf["inputs"]["sensors"]["right"]["image"]["main_file"]
bands_left = list(conf["inputs"]["sensors"]["left"]["image"]["bands"].keys())
bands_right = list(conf["inputs"]["sensors"]["right"]["image"]["bands"].keys())
dense_matching_mccnn_application.corr_config = (
    dense_matching_mccnn_application.loader.check_conf(
        corr_cfg, left_image, right_image, bands_left, bands_right
    )
)

In [ ]:
epipolar_disparity_map_mccnn = dense_matching_mccnn_application.run(
    new_epipolar_image_left,
    new_epipolar_image_right,
    local_tile_optimal_size_fun,
    orchestrator=cars_orchestrator,
    disp_range_grid=disp_range_grid,
    texture_bands=texture_bands_indices,
)

In [ ]:
data_disparity_mccnn = get_full_data(epipolar_disparity_map_mccnn, "disp")
show_data(data_disparity_mccnn)

## Compute two DSM and compare them

One  from disparity map computed by Census similarity measure and the other from disparity map from MC-CNN similarity measure

In [ ]:
conf_outlier_removal_small_components = {"method": "small_components", "activated": True}
pc_outlier_removal_small_comp_application = Application("point_cloud_outlier_removal", cfg=conf_outlier_removal_small_components)

In [ ]:
conf_outlier_removal_statistical = {"method": "statistical", "activated": True}
pc_outlier_removal_stats_application = Application("point_cloud_outlier_removal", cfg=conf_outlier_removal_statistical)

In [ ]:
pc_fusion_application = Application("point_cloud_fusion")

In [ ]:
conf_rasterization = { 
    "method": "simple_gaussian",
    "dsm_radius": 3,
    "sigma": 0.3
}
rasterization_application = Application("point_cloud_rasterization", cfg=conf_rasterization)

 Compute epsg

In [ ]:
epsg = preprocessing.compute_epsg(
    sensor_image_left, 
    sensor_image_right,
    grid_left,
    corrected_grid_right,
    geom_plugin_with_dem_and_geoid,
    disp_min=np.min(disp_range_grid[0, 0]["disp_min_grid"].values),
    disp_max=np.max(disp_range_grid[0, 0]["disp_max_grid"].values)
)

### Triangulation : triangulate matches

From census disparity map

In [ ]:
epipolar_point_cloud_census = triangulation_application.run(
    sensor_image_left,
    sensor_image_right,
    corrected_grid_left,
    corrected_grid_right,
    epipolar_disparity_map_census,
    geom_plugin_without_dem_and_geoid,
    new_epipolar_image_left,
    epsg,
    orchestrator=cars_orchestrator,
    uncorrected_grid_right=grid_right,
    geoid_path=updated_inputs_conf[sens_cst.INITIAL_ELEVATION][sens_cst.GEOID],
)

From mccnn disparity map

In [ ]:
epipolar_point_cloud_mccnn = triangulation_application.run(
    sensor_image_left,
    sensor_image_right,
    corrected_grid_left,
    corrected_grid_right,
    epipolar_disparity_map_mccnn,
    geom_plugin_without_dem_and_geoid,
    new_epipolar_image_left,
    epsg,
    orchestrator=cars_orchestrator,
    uncorrected_grid_right=grid_right,
    geoid_path=updated_inputs_conf[sens_cst.INITIAL_ELEVATION][sens_cst.GEOID],
)

 #### Compute terrain bounding box

In [ ]:
current_terrain_roi_bbox, _ = preprocessing.compute_terrain_bbox(
    sensor_image_left,
    sensor_image_right,
    new_epipolar_image_left,
    grid_left,
    corrected_grid_right,
    epsg,
    geom_plugin_with_dem_and_geoid,
    resolution=0.5,
    disp_min=np.min(disp_range_grid[0, 0]["disp_min_grid"].values),
    disp_max=np.max(disp_range_grid[0, 0]["disp_max_grid"].values),
    orchestrator=cars_orchestrator
)
terrain_bounds, optimal_terrain_tile_width = preprocessing.compute_terrain_bounds(
    [current_terrain_roi_bbox],
    resolution=0.5
)

#### Transform point cloud to terrain point cloud

From census disparity map

In [ ]:
merged_point_clouds_census = pc_fusion_application.run(
    [epipolar_point_cloud_census],
    terrain_bounds,
    epsg,
    orchestrator=cars_orchestrator,
    margins=rasterization_application.get_margins(0.5),
    optimal_terrain_tile_width=optimal_terrain_tile_width
)

From mccnn disparity map

In [ ]:
merged_point_clouds_mccnn = pc_fusion_application.run(
    [epipolar_point_cloud_mccnn],
    terrain_bounds,
    epsg,
    orchestrator=cars_orchestrator,
    margins=rasterization_application.get_margins(0.5),
    optimal_terrain_tile_width=optimal_terrain_tile_width
)

### Point Cloud Outlier Removal : remove points with the small component removal method

From census disparity map

In [ ]:
filtered_sc_merged_point_clouds_census = pc_outlier_removal_small_comp_application.run(
    merged_point_clouds_census,
    orchestrator=cars_orchestrator,
)

From mccnn disparity map

In [ ]:
filtered_sc_merged_point_clouds_mccnn = pc_outlier_removal_small_comp_application.run(
    merged_point_clouds_mccnn,
    orchestrator=cars_orchestrator,
)

### Point Cloud Outlier Removal: remove points with statistical removal method

From census disparity map

In [ ]:
filtered_stats_merged_point_clouds_census = pc_outlier_removal_stats_application.run(
    filtered_sc_merged_point_clouds_census,
    orchestrator=cars_orchestrator,
)

From mccnn disparity map

In [ ]:
filtered_stats_merged_point_clouds_mccnn = pc_outlier_removal_stats_application.run(
    filtered_sc_merged_point_clouds_mccnn,
    orchestrator=cars_orchestrator,
)

### Rasterization : rasterize point cloud

From census disparity map

In [ ]:
dsm_census = rasterization_application.run(
    filtered_stats_merged_point_clouds_census,
    epsg,
    resolution=0.5,
    orchestrator=cars_orchestrator
)

From mccnn disparity map

In [ ]:
dsm_mccnn = rasterization_application.run(
    filtered_stats_merged_point_clouds_mccnn,
    epsg,
    resolution=0.5,
    orchestrator=cars_orchestrator
)

### Show DSM


From census disparity map

In [ ]:
data_dsm_census = get_full_data(dsm_census, "hgt")
show_data(data_dsm_census, mode="dsm")

From mccnn disparity map

In [ ]:
data_dsm_mccnn = get_full_data(dsm_mccnn, "hgt")
show_data(data_dsm_mccnn, mode="dsm")

### Show ortho image

From census disparity map

In [ ]:
data_ortho_census = get_full_data(dsm_census, "img")
show_data(data_ortho_census, mode='image')

From mccnn disparity map

In [ ]:
data_ortho_mccnn = get_full_data(dsm_mccnn, "img")
show_data(data_dsm_mccnn, mode='image')